![image info](https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/images/banner_1.png)

# Proyecto 1 - Predicción de precios de vehículos usados

En este proyecto podrán poner en práctica sus conocimientos sobre modelos predictivos basados en árboles y ensambles, y sobre la disponibilización de modelos. Para su desasrrollo tengan en cuenta las instrucciones dadas en la "Guía del proyecto 1: Predicción de precios de vehículos usados".

**Entrega**: La entrega del proyecto deberán realizarla durante la semana 4. Sin embargo, es importante que avancen en la semana 3 en el modelado del problema y en parte del informe, tal y como se les indicó en la guía.

Para hacer la entrega, deberán adjuntar el informe autocontenido en PDF a la actividad de entrega del proyecto que encontrarán en la semana 4, y subir el archivo de predicciones a la [competencia de Kaggle](https://www.kaggle.com/competitions/miad2024-12-prediccion-precio-vehiculos).

## Datos para la predicción de precios de vehículos usados

En este proyecto se usará el conjunto de datos de Car Listings de Kaggle, donde cada observación representa el precio de un automóvil teniendo en cuenta distintas variables como: año, marca, modelo, entre otras. El objetivo es predecir el precio del automóvil. Para más detalles puede visitar el siguiente enlace: [datos](https://www.kaggle.com/jpayne/852k-used-car-listings).

## Ejemplo predicción conjunto de test para envío a Kaggle

En esta sección encontrarán el formato en el que deben guardar los resultados de la predicción para que puedan subirlos a la competencia en Kaggle.

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
import joblib

def load_data():
    # Carga de datos
    dataTraining = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/datasets/dataTrain_carListings.zip')
    dataTesting = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/datasets/dataTest_carListings.zip', index_col=0)
    return dataTraining, dataTesting

def preprocess_data(data):
    # Copia de datos para preprocesamiento
    data_processed = data.copy()
    # Ajuste del precio y millaje
    data_processed['Price'] = np.log(data_processed['Price'])
    data_processed['Mileage'] = np.log(data_processed['Mileage'])
    return data_processed

def encode_features(X):
    # Inicialización de encoders
    le_State = LabelEncoder()
    le_Make = LabelEncoder()
    le_Model = LabelEncoder()
    
    # Aplicación de encoders
    X['State_encoded'] = le_State.fit_transform(X['State'])
    X['Make_encoded'] = le_Make.fit_transform(X['Make'])
    X['Model_encoded'] = le_Model.fit_transform(X['Model'])

    # Guardar encoders para uso futuro
    joblib.dump(le_State, 'le_State.pkl')
    joblib.dump(le_Make, 'le_Make.pkl')
    joblib.dump(le_Model, 'le_Model.pkl')

    return X[['Year', 'Mileage', 'State_encoded', 'Make_encoded', 'Model_encoded']], le_State, le_Make, le_Model

def train_model(X, y):
    # Entrenamiento del modelo RandomForest
    reg_RF = RandomForestRegressor(max_features=4, n_estimators=100, max_depth=19, random_state=1, min_samples_split=17, n_jobs=-1)
    reg_RF.fit(X, y)

    # Guardar el modelo entrenado
    joblib.dump(reg_RF, 'model_rf.pkl', compress=6)
    return reg_RF

def predict_price(X, model):
    # Predicción de precios
    return model.predict(X)

def load_model_and_encoders():
    # Cargar modelo y encoders guardados
    model = joblib.load('model_rf.pkl')
    le_State = joblib.load('le_State.pkl')
    le_Make = joblib.load('le_Make.pkl')
    le_Model = joblib.load('le_Model.pkl')
    return model, le_State, le_Make, le_Model

# Ejecución del script para entrenamiento y prueba si se ejecuta como principal
if __name__ == "__main__":
    data_train, data_test = load_data()
    data_train_processed = preprocess_data(data_train)
    X_train, le_State, le_Make, le_Model = encode_features(data_train_processed[['Year', 'Mileage', 'State', 'Make', 'Model']])
    y_train = data_train_processed['Price']

    model = train_model(X_train, y_train)
    
    # Simulación de carga de modelo y encoders para predicción
    model_loaded, le_State_loaded, le_Make_loaded, le_Model_loaded = load_model_and_encoders()
    X_test, _, _, _ = encode_features(data_test[['Year', 'Mileage', 'State', 'Make', 'Model']])
    y_pred = predict_price(X_test, model_loaded)
    print("Predicciones:", y_pred)

Predicciones: [ 9.33667985 10.17184192  9.7132689  ...  9.64727679  9.50757267
  9.58924425]


In [5]:
from flask import Flask, request, jsonify
import pandas as pd
import joblib

app = Flask(__name__)

# Carga del modelo y encoders
model = joblib.load('model_rf.pkl')
le_State = joblib.load('le_State.pkl')
le_Make = joblib.load('le_Make.pkl')
le_Model = joblib.load('le_Model.pkl')

def preprocess_data(df):
    # Aquí deberías incluir la lógica de preprocesamiento aplicada antes del entrenamiento
    df['Mileage'] = np.log(df['Mileage'])
    return df

def encode_features(df):
    df['State_encoded'] = le_State.transform(df['State'])
    df['Make_encoded'] = le_Make.transform(df['Make'])
    df['Model_encoded'] = le_Model.transform(df['Model'])
    return df[['Year', 'Mileage', 'State_encoded', 'Make_encoded', 'Model_encoded']]

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json(force=True)
    df = pd.DataFrame(data, index=[0])

    # Preprocesamiento y codificación de características
    df_processed = preprocess_data(df)
    df_encoded = encode_features(df_processed)
    
    # Predicción
    prediction = model.predict(df_encoded)
    
    # Devolución de la predicción
    return jsonify({'prediction': prediction.tolist()})

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.3:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

c:\Users\edgar\.pyenv\pyenv-win\versions\3.10.5\lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
